## 1.Prepare on Oulu dataset
__[Dataset Description](http://www.ee.oulu.fi/~gyzhao/Download/Databases/NIR_VL_FED/Description.pdf)__

(1) Extract face from the original video and randomly select 7 frames as a new video.

* input_folder_path = '/.../Oulu-CASIA/A_OriginalVideo/VL/S'
* output_folder_path = '/.../oulu_face_extracted'
* ExtractFaceMMOD_Oulu7Frames.py

(2)Transform 7 frames to a tensor, and save as a sampleLib list. 

* input_folder_path = '/.../oulu_face_extracted'
* output_path = '/.../oulu_trans_video_list.pkl'


In [4]:
import cv2
import os
import dlib
from tqdm import tqdm
from time import time
import numpy as np
import threading
from random import sample


class FaceExtractor():
    def detect_face(self, frame):
       
        """
        Args:
            net: face detector
            frame: image for face detection
        Returns:
            flag: whether there is a face
            face_patch: cropped face region
        """
        inHeight = 300

        dnnFaceDetector = dlib.cnn_face_detection_model_v1("./models/mmod_human_face_detector.dat")
        
        # Load face detector and process the frame
        frameDlibMMOD = frame.copy()
        frameHeight = frameDlibMMOD.shape[0]
        frameWidth = frameDlibMMOD.shape[1]
        inWidth = int((frameWidth / frameHeight)*inHeight)
        scaleHeight = frameHeight / inHeight
        scaleWidth = frameWidth / inWidth
        
        frameDlibMMODSmall = cv2.resize(frameDlibMMOD, (inWidth, inHeight))
        frameDlibMMODSmall = cv2.cvtColor(frameDlibMMODSmall, cv2.COLOR_BGR2RGB)
        faceRects = dnnFaceDetector(frameDlibMMODSmall, 0)
        # Traverse the detection results
    
        """
        # extract face region from background
        face_patch = frameDlibMMO[y1:y2, x1:x2, :]
    
        """
        face_patch = None
        for faceRect in faceRects:
            if  faceRect is not None:
                cvRect = [int(faceRect.rect.left()*scaleWidth), int(faceRect.rect.top()*scaleHeight),                  int(faceRect.rect.right()*scaleWidth), int(faceRect.rect.bottom()*scaleHeight) ]
                
                x1 = cvRect[1]
                y1 = cvRect[0]
                x2 = cvRect[3]
                y2 = cvRect[2]
                
                # extract face region from background
                face_patch = frameDlibMMOD[x1:x2, y1:y2, :]
                return face_patch
                
                
    def extract_face(self, input_video, output_folder,  resolution):
        """
        Args:
            input_video: path of given video
            output_path: target dir to save extracted video
            conf_thresh: threshold for face detection
            resolution: frame size of extracted face region
        """
        # get save path 
        video_name = os.path.split(input_video.split('.')[0])[-1]
        save_path = os.path.join(output_folder, video_name+'.avi')
        # build video writer
        fourcc = cv2.VideoWriter_fourcc(*'MJPG')
        VideoWriter = cv2.VideoWriter(filename=save_path,fourcc=fourcc,fps=30,frameSize=resolution, isColor=True)
        # Load video 
        cap = cv2.VideoCapture(input_video)
        video_len = (cap.get(7))
        if int(video_len) == 7:
            ls_7frame = [0,1,2,3,4,5,6]
        else:
            population = range(int(cap.get(7))-1)
            ls_7frame = (sample(population,7))  # order num for 7 frames
            ls_7frame = sorted(ls_7frame)
        print(ls_7frame)
        for i in range(int(cap.get(7))):
            ret, frame = cap.read()
            face = self.detect_face(frame)
#             print(face)
            
            # resize and write frame into video, if there is a face
            if face is not None: 
                
                try:
                    face = cv2.resize(face, resolution)
                    
                    if i in ls_7frame:
                        print(i)
                        VideoWriter.write(face)
                except:
                    pass
        cap.release()
        VideoWriter.release()
        
        return None
    
    
    
    
    def run(self, input_video, output_dir):
        """
        Args:
            video_list: paths of input videos
            output_dir: target folder to save extracted videos
        """
        # Extract face region from each video 
#         for i in video_list:
        self.extract_face(input_video, output_folder=output_dir,resolution=(224,224))
            
        return None


In [ ]:
if __name__=="__main__":

    input_folder_path = '/.../Oulu-CASIA/A_OriginalVideo/VL'
    output_folder_path = '/.../oulu_face_extracted'

    face_extractor = FaceExtractor()


    for root,dirs,files in os.walk(input_folder_path):
        for file in files:
            if file[8] == 'S':
                if int(file[5:7]) == 58:
                    print(file)
                    input_video = os.path.join(root, file)
                    face_extractor.run(input_video,output_folder_path)

            


### (2)Transform 7 frames to a tensor, and save as a sampleLib list. 

* input_folder_path = '/.../oulu_face_extracted'
* output_path = '/.../oulu_trans_video_list.pkl'


In [1]:
import torch
import cv2
import os
from torchvision import transforms

In [2]:
def video_tensor(input_video):
    transform=transforms.Compose([  transforms.ToPILImage(),
    transforms.Resize(256),  
                                transforms.CenterCrop(224),  # crop to 224
                                transforms.ToTensor()])
    tensor_list = []
    cap = cv2.VideoCapture(input_video)
    for i in range(int(cap.get(7))):
        ret, frame = cap.read()
        frame_trans = transform(frame)
        frame_trans = torch.unsqueeze(frame_trans, 0)
        tensor_list.append(frame_trans)
    video_tensor = torch.cat(tensor_list,dim=0) 
    return video_tensor

In [3]:
if __name__=="__main__":

    input_folder_path = '/.../oulu_face_extracted'
    label_dict = {'A':0,'D':3,'F':4,'H':5,'1':1,'2':2}

    video_list = []
    
    for root,dirs,files in os.walk(input_folder_path):
        for file in files:
            input_video = os.path.join(root, file)
            tensor_list  = video_tensor(input_video)
            label_class = input_video[-5]
            label = label_dict[label_class]
            if int(tensor_list.shape[0]) !=7:
                print(file)
            video_list.append([tensor_list,label])
    torch.save(video_list, '/.../oulu_trans_video_tensor_list.pkl')
    
